In [2]:
from sklearn.datasets import twenty_newsgroups

In [3]:
data = twenty_newsgroups.fetch_20newsgroups()

In [4]:
data.data[10]

'From: irwin@cmptrc.lonestar.org (Irwin Arnstein)\nSubject: Re: Recommendation on Duc\nSummary: What\'s it worth?\nDistribution: usa\nExpires: Sat, 1 May 1993 05:00:00 GMT\nOrganization: CompuTrac Inc., Richardson TX\nKeywords: Ducati, GTS, How much? \nLines: 13\n\nI have a line on a Ducati 900GTS 1978 model with 17k on the clock.  Runs\nvery well, paint is the bronze/brown/orange faded out, leaks a bit of oil\nand pops out of 1st with hard accel.  The shop will fix trans and oil \nleak.  They sold the bike to the 1 and only owner.  They want $3495, and\nI am thinking more like $3K.  Any opinions out there?  Please email me.\nThanks.  It would be a nice stable mate to the Beemer.  Then I\'ll get\na jap bike and call myself Axis Motors!\n\n-- \n-----------------------------------------------------------------------\n"Tuba" (Irwin)      "I honk therefore I am"     CompuTrac-Richardson,Tx\nirwin@cmptrc.lonestar.org    DoD #0826          (R75/6)\n-------------------------------------------

In [5]:
data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
data.target[10]

8

In [7]:
categories = ['alt.atheism', 'sci.med', 'soc.religion.christian', 'misc.forsale']

In [8]:
twenty_train = twenty_newsgroups.fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [9]:
twenty_test = twenty_newsgroups.fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

In [10]:
len(twenty_train.data)

2258

In [11]:
len(twenty_test.data)

1503

In [12]:
twenty_train.target_names

['alt.atheism', 'misc.forsale', 'sci.med', 'soc.religion.christian']

In [13]:
import re

In [14]:
re.sub('hi','','hi hello')

' hello'

In [15]:
#re.sub('From: [\w.\s]+','',twenty_train.data[1])

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [17]:
cv = CountVectorizer(stop_words='english')

In [26]:
pipe = Pipeline([
    ('cv',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('mnb',MultinomialNB())
])

In [19]:
pipe.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
     steps=[('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        st...inear_tf=False, use_idf=True)), ('mnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [20]:
pipe.predict(['love all'])

array([3], dtype=int64)

In [21]:
pipe.predict(['i am doing well these days not much sick'])

array([2], dtype=int64)

In [22]:
pipe.predict(['is there any discount on products sale'])

array([1], dtype=int64)

In [23]:
pipe.predict(['i am fine these days'])

array([2], dtype=int64)

In [24]:
pipe.score(twenty_test.data,twenty_test.target)

0.8995342648037259

In [28]:
help(MultinomialNB)

Help on class MultinomialNB in module sklearn.naive_bayes:

class MultinomialNB(BaseDiscreteNB)
 |  Naive Bayes classifier for multinomial models
 |  
 |  The multinomial Naive Bayes classifier is suitable for classification with
 |  discrete features (e.g., word counts for text classification). The
 |  multinomial distribution normally requires integer feature counts. However,
 |  in practice, fractional counts such as tf-idf may also work.
 |  
 |  Read more in the :ref:`User Guide <multinomial_naive_bayes>`.
 |  
 |  Parameters
 |  ----------
 |  alpha : float, optional (default=1.0)
 |      Additive (Laplace/Lidstone) smoothing parameter
 |      (0 for no smoothing).
 |  
 |  fit_prior : boolean, optional (default=True)
 |      Whether to learn class prior probabilities or not.
 |      If false, a uniform prior will be used.
 |  
 |  class_prior : array-like, size (n_classes,), optional (default=None)
 |      Prior probabilities of the classes. If specified the priors are not
 |   

In [29]:
params = {
    'cv__stop_words':['english',['good','medicine','god']],
    'tfidf__use_idf':[True,False],
    'mnb__fit_prior':[True,False]
}

In [30]:
from sklearn.grid_search import GridSearchCV

In [34]:
grid = GridSearchCV(pipe, param_grid=params, cv=5, n_jobs=-1)

In [35]:
grid.fit(twenty_train.data, twenty_train.target)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_a...inear_tf=False, use_idf=True)), ('mnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'cv__stop_words': ['english', ['good', 'medicine', 'god']], 'tfidf__use_idf': [True, False], 'mnb__fit_prior': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [36]:
grid.best_params_

{'cv__stop_words': 'english', 'mnb__fit_prior': False, 'tfidf__use_idf': True}